<h1>Covid Data Prep</h1>
<h3>Feature Engineering</h3>
<p>The notebook below takes the Covid-19 df and prepares it to use in Deep Learning Notebooks</p>
<br>
<p>The raw data consists of the following feature</p>
<ol>
    <li>dateRep</li>
    <li style="color:red;">day</li>
    <li style="color:red;">month</li>
    <li style="color:red;">year</li>
    <li>cases</li>
    <li>deaths</li>
    <li>countriesAndTerritories</li>
    <li style="color:red;">geoId</li>
    <li style="color:red;">countryterritorycode</li>
    <li>popData2018</li>
    <li>continentExp</li>
</ol>
<br>
</p>Feature names in red will be remove as they were defined as adding little information to the model. The remaining feautures will be evaluated through the notebook below.</p>

<h5>Import dependancies</h5>
<ul>
    <li>pandas: feature extrapolation and extraction and creation</li>
    <li>numpy: numerical data manipluation</li>
    <li>os: interaction with the operating system</li>
    <li>seaborn: plotting library</li>
    <li>sklearn.model_selection.train_test_split: spliting the data into the various data sets (train, test and validation)</li>
</ul>

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

</h2>Read in raw data</h2>

<p>Change to the relevant directory and read in the csv</p>

The csv will need some preprocessing

In [10]:
data_folder = os.path.join(os.getcwd(), 'Datasets')

<p>Now read the file. No Line should need skipping.</p>

In [143]:
# Import and format dataframe
covid19_df = pd.read_csv(os.path.join(data_folder, 'COVID-19_Cases_Worldwide_2020_08_21'), engine='python')
covid19_df.head(2)

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
0,20/08/2020,20,8,2020,160,8,Afghanistan,AF,AFG,38041757.0,Asia,2.268560
1,19/08/2020,19,8,2020,0,0,Afghanistan,AF,AFG,38041757.0,Asia,2.024092


In [144]:
# Columns to drop
drop_columns = ['geoId', 'day', 'month', 'year', 'countryterritoryCode']
# Create a 'datetime' column based on the dates
covid19_df['dateRep'] = pd.to_datetime(covid19_df['dateRep'], dayfirst=True)
# Drop the columns that add no value
covid19_df.drop(columns=drop_columns, inplace=True)

In [145]:
# Sort the table by the date
covid19_df.sort_values(by=['dateRep'], ascending=True, inplace=True)
# Create a cumulative sum of covid cases and deaths
covid19_df['Cum_Cases'] = covid19_df.groupby("countriesAndTerritories")['cases'].cumsum()
covid19_df['Cum_Deaths'] = covid19_df.groupby("countriesAndTerritories")['deaths'].cumsum()

In [146]:
# Create column for days since x deaths
covid19_df['flag'] = np.where(covid19_df['Cum_Cases'] > 100, 1, 0) # calculate globaly as its a true false
# groupby again creating a unique dataframe for each country, and applying a cumulative sum to the "flag" column
covid19_df['flag'] = covid19_df.loc[covid19_df['Cum_Cases'] > 100].groupby("countriesAndTerritories")['flag'].cumsum()

<p>We don't want Nan's so we replace them in the next few cells below</p>

In [147]:
for i in covid19_df.columns:
    frac_null = covid19_df[i].isna().sum() /len(covid19_df)
    print(i, ':', frac_null)

dateRep : 0.0
cases : 0.0
deaths : 0.0
countriesAndTerritories : 0.0
popData2019 : 0.001709082169466179
continentExp : 0.0
Cumulative_number_for_14_days_of_COVID-19_cases_per_100000 : 0.07426496114508506
Cum_Cases : 0.0
Cum_Deaths : 0.0
flag : 0.3319358026010094


In [148]:
top_count = covid19_df.groupby('countriesAndTerritories')['deaths'].sum().sort_values(ascending=False).iloc[:20]
top_count = list(top_count.keys())
top_count

['United_States_of_America',
 'Brazil',
 'Mexico',
 'India',
 'United_Kingdom',
 'Italy',
 'France',
 'Spain',
 'Peru',
 'Iran',
 'Russia',
 'Colombia',
 'South_Africa',
 'Chile',
 'Belgium',
 'Germany',
 'Canada',
 'Indonesia',
 'Pakistan',
 'Netherlands']

In [58]:
# Trim the dataframe to just the 20 most affected countries
covid19_df = covid19_df[covid19_df['countriesAndTerritories'].isin(top_count)]
# Fill the empty 'flag' rows
covid19_df.fillna(0, inplace=True)

In [229]:
mobility_df = pd.read_csv(os.path.join(data_folder, 'Apple_Mobility' , 'applemobilitytrends-2020-08-19.csv'), engine='python')

In [230]:
# Take only the country data, drop regional data
mobility_df = mobility_df.loc[mobility_df['geo_type'] == 'country/region']
# Columns to drop
drop_columns = ['alternative_name', 'sub-region', 'country', 'geo_type']
# Drop the columns that add no value
mobility_df.drop(columns=drop_columns, inplace=True)
mob_list = mobility_df.columns.to_list()[2:]

In [231]:
mobility_df = mobility_df.melt(id_vars=['transportation_type', 'region'])
# Format the strings so the countries match
mobility_df = mobility_df.replace(' ', '_', regex=True)
mobility_df = mobility_df.replace('United_States', 'United_States_of_America', regex=True)
mobility_df = mobility_df.replace('Republic_of_Korea', 'South_Korea', regex=True)
# Create a 'datetime' column based on the dates
mobility_df['variable'] = pd.to_datetime(mobility_df['variable'], dayfirst=True)
mobility_df.head(2)

,transportation_type,region,variable,value
0,driving,Albania,2020-01-13,100.0
1,walking,Albania,2020-01-13,100.0


In [ ]:
for country in top_count1:
    country_len = len(country_df_2.loc[(country_df_2['countriesAndTerritories'] == country) & (country_df_2['flag'] > 0)])
    blank_df = pd.DataFrame(columns=['driving', 'walking', 'transit', 'region', 'dateRep'], index=np.arange(len(mob_list))
    for i in range(len(country_df_mean) - country_len):
        date_value_max = pd.DateOffset(i + 1) + country_df_2.loc[(country_df_2['countriesAndTerritories'] == country)]['dateRep'].max()
        pop_value = country_df_2.loc[(country_df_2['dateRep'] == (country_df_2.loc[(country_df_2['countriesAndTerritories'] == country)]['dateRep'].max())) & (country_df_2['countriesAndTerritories'] == country)]['popData2018'].iloc[0]
        #continentExp_value = country_df_2.loc[(country_df_2['dateRep'] == (country_df_2.loc[(country_df_2['countriesAndTerritories'] == country)]['dateRep'].max())) & (country_df_2['countriesAndTerritories'] == country)]['continentExp'].iloc[0]
        deathpred_value = country_df_2.loc[(country_df_2['dateRep'] == (country_df_2.loc[(country_df_2['countriesAndTerritories'] == country)]['dateRep'].max())) & (country_df_2['countriesAndTerritories'] == country)]['5D_MA_deaths'].iloc[0]
        #deathpred_value = country_df_2.loc[(country_df_2['dateRep'] == (country_df_2.loc[(country_df_2['countriesAndTerritories'] == country)]['dateRep'].max())) & (country_df_2['countriesAndTerritories'] == country)]['deaths'].iloc[0]
        cumprod_value = 1
        blank_df = blank_df.append({'flag': (i + country_len + 1), 'countriesAndTerritories': country, 'dateRep': date_value_max, 'popData2018': pop_value, 'cum_7D_MA_deaths_global': 1, '7D_DeathPred': deathpred_value}, ignore_index=True) # 'continentExp': continentExp_value,
    country_df_2 = pd.concat([country_df_2, blank_df])
country_df_2.loc[country_df_2['countriesAndTerritories'] == 'United_Kingdom'].tail()

In [222]:
top_count1 = mobility_df.groupby('region')['value'].sum().sort_values(ascending=False).iloc[:]
top_count1 = list(top_count1.keys())
top_count1

['Estonia',
 'Croatia',
 'Norway',
 'Sweden',
 'Finland',
 'Denmark',
 'Japan',
 'Germany',
 'Taiwan',
 'Slovakia',
 'Switzerland',
 'Czech_Republic',
 'Belgium',
 'Canada',
 'Netherlands',
 'United_States_of_America',
 'France',
 'Lithuania',
 'Latvia',
 'Slovenia',
 'Italy',
 'Ireland',
 'Spain',
 'United_Kingdom',
 'Luxembourg',
 'Russia',
 'Ukraine',
 'New_Zealand',
 'Australia',
 'Mexico',
 'Bulgaria',
 'Greece',
 'Iceland',
 'Poland',
 'Brazil',
 'Austria',
 'Turkey',
 'Hungary',
 'Vietnam',
 'Albania',
 'Singapore',
 'Romania',
 'Portugal',
 'Malaysia',
 'Egypt',
 'Israel',
 'Serbia',
 'Saudi_Arabia',
 'United_Arab_Emirates',
 'Thailand',
 'Indonesia',
 'Colombia',
 'Philippines',
 'South_Africa',
 'Cambodia',
 'India',
 'South_Korea',
 'Morocco',
 'Hong_Kong',
 'Uruguay',
 'Chile',
 'Argentina',
 'Macao']

In [71]:
covid_df = covid19_df.merge(mobility_df, left_on['countriesAndTerritories'])

KeyError: 'Level transportation_type not found'

<p>Above we filtered out countries not in the 'top_count' list</p>

<p>Below we export the .csv as a master dataframe. We then do the final processing stages for the train and test datasets</p>

In [12]:
# Check if the folder 'data_export' exists and if not create it.
folder_create = os.path.exists(os.path.join(data_folder, "data_export"))
if folder_create is False:
    os.mkdir(os.path.join(data_folder, "data_export"))
# Export the polished dataframe to re-import after modelling.
covid19_df.to_csv(os.path.join(data_folder, "data_export", "covid19_df.csv"))

In [13]:
# 'get_dummies' creates a new column for each country that is populated with either a 1 or a 0
df_train = pd.get_dummies(data=covid19_df, columns=["countriesAndTerritories"])
# df_y is target to predict, in this case 'deaths'
df_y = df_train[['deaths', 'dateRep']]
# df_train contains the columns we will use to predict 'deaths'
df_train.drop(columns=['cases', 'deaths', 'Cum_Cases', 'Cum_Deaths'], inplace=True)

In [14]:
data_path = os.path.join(data_folder, "data")
# Check if the 'data' folder exists, if not create it
folder_create = os.path.exists(data_path)
if folder_create is False:
    os.mkdir(data_path)
    # Create 'train' and 'test' folders if the data folder does not exist.
    os.mkdir(os.path.join(data_path, "train"))
    os.mkdir(os.path.join(data_path, "test"))

In [15]:
# Instead of randomly splitting the data we will select a date to test 'blind' from
date_slice = '2020-04-23'

In [16]:
# Data split according to the date. 'train'=before date, 'test'=after.
X_train = df_train.loc[df_train['dateRep'] < date_slice]
X_test = df_train.loc[df_train['dateRep'] >= date_slice]
y_train = df_y.loc[df_y['dateRep'] < date_slice]
y_test = df_y.loc[df_y['dateRep'] >= date_slice]

In [17]:
# Export these four dataframes for later.
X_train.to_csv(os.path.join(data_path, "train", "train_x.csv"))
y_train.to_csv(os.path.join(data_path, "train", "train_y.csv"))
X_test.to_csv(os.path.join(data_path, "test", "test_x.csv"))
y_test.to_csv(os.path.join(data_path, "test", "test_y.csv"))